# Example of `InferenceData` schema in PyMC3
The description of the `InferenceData` structure can be found [here](schema.html).

In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = data.time.values
slack_comments = data.comments.values
commits = data.commits.values
names = data.index.values
N = len(names)
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = np.array([3.6, 5.1])

In [4]:
dims={
    "slack_comments": ["developer"],
    "github_commits": ["developer"],
    "time_since_joined": ["developer"],
}
with pm.Model() as model:
    time_since_joined = pm.Data("time_since_joined", time)
    
    b_sigma = pm.HalfNormal('b_sigma', sd=300)
    c_sigma = pm.HalfNormal('c_sigma', sd=6)
    b0 = pm.Normal("b0", mu=0, sd=200)
    b1 = pm.Normal("b1", mu=0, sd=200)
    c0 = pm.Normal("c0", mu=0, sd=10)
    c1 = pm.Normal("c1", mu=0, sd=10)
    
    pm.Normal("slack_comments", mu=b0 + b1 * time_since_joined, sigma=b_sigma, observed=slack_comments)
    pm.Normal("github_commits", mu=c0 + c1 * time_since_joined, sigma=c_sigma, observed=commits)
    
    trace = pm.sample(400, chains=4)
    posterior_predictive = pm.sample_posterior_predictive(trace)
    prior = pm.sample_prior_predictive(150)
    idata_pymc3 = az.from_pymc3(
        trace,
        prior=prior,
        posterior_predictive=posterior_predictive,
        coords={"developer": names},
        dims=dims
    )

Only 400 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c1, c0, b1, b0, c_sigma, b_sigma]


There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [5]:
with model:
    pm.set_data({"time_since_joined": candidate_devs_time})
    predictions = pm.sample_posterior_predictive(trace)
    az.from_pymc3_predictions(
        predictions, 
        idata_orig=idata_pymc3, 
        inplace=True,
        coords={"developer": candidate_devs},
        dims=dims,
    )

In [6]:
idata_pymc3

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> log_likelihood
	> constant_data
	> predictions
	> predictions_constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [7]:
idata_pymc3.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 400)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    b0       (chain, draw) float64 -196.4 609.0 343.5 ... -123.3 -115.5 -236.1
    b1       (chain, draw) float64 1.574e+03 1.642e+03 ... 1.58e+03 1.591e+03
    c0       (chain, draw) float64 3.555 1.61 2.323 0.3893 ... 8.996 7.099 3.282
    c1       (chain, draw) float64 5.601 6.023 5.276 5.603 ... 3.905 4.881 5.057
    b_sigma  (chain, draw) float64 1.775e+03 1.586e+03 ... 1.526e+03 1.597e+03
    c_sigma  (chain, draw) float64 5.901 7.405 9.769 5.567 ... 7.551 5.494 3.619
Attributes:
    created_at:                 2020-01-26T18:38:57.295717
    inference_library:          pymc3
    inference_library_version:  3.8

In [8]:
idata_pymc3.sample_stats

<xarray.Dataset>
Dimensions:           (chain: 4, draw: 400)
Coordinates:
  * chain             (chain) int64 0 1 2 3
  * draw              (draw) int64 0 1 2 3 4 5 6 ... 393 394 395 396 397 398 399
Data variables:
    energy            (chain, draw) float64 145.4 146.7 146.0 ... 149.4 146.3
    tune              (chain, draw) bool True False False ... False False False
    diverging         (chain, draw) bool False False False ... False False False
    tree_size         (chain, draw) float64 7.0 7.0 7.0 7.0 ... 7.0 7.0 15.0 7.0
    lp                (chain, draw) float64 -141.7 -144.4 ... -140.4 -141.9
    depth             (chain, draw) int64 3 3 3 3 4 3 4 4 4 ... 3 3 3 4 3 3 4 3
    step_size         (chain, draw) float64 0.3876 0.3876 ... 0.2923 0.2923
    step_size_bar     (chain, draw) float64 0.3551 0.3551 ... 0.3768 0.3768
    mean_tree_accept  (chain, draw) float64 0.8259 0.8353 1.0 ... 0.9778 0.6642
    max_energy_error  (chain, draw) float64 0.6135 0.4614 ... -0.6488 -0.7487
    energy_error      (chain, draw) float64 0.2204 0.3375 ... -0.6488 -0.7487
Attributes:
    created_at:                 2020-01-26T18:38:57.299451
    inference_library:          pymc3
    inference_library_version:  3.8

In [9]:
idata_pymc3.log_likelihood

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 -8.46 -8.516 ... -8.455
    github_commits  (chain, draw, developer) float64 -2.897 -3.076 ... -2.314
Attributes:
    created_at:                 2020-01-26T18:38:57.543608
    inference_library:          pymc3
    inference_library_version:  3.8

In [10]:
idata_pymc3.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 5.038e+03 ... 2.947e+04
    github_commits  (chain, draw, developer) float64 36.92 44.14 ... 60.22 96.89
Attributes:
    created_at:                 2020-01-26T18:38:57.545285
    inference_library:          pymc3
    inference_library_version:  3.8

In [11]:
idata_pymc3.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) float64 7.5e+03 1.01e+04 ... 2.52e+04 2.75e+04
    github_commits  (developer) float64 25.0 32.0 49.0 66.0 96.0
Attributes:
    created_at:                 2020-01-26T18:38:57.551445
    inference_library:          pymc3
    inference_library_version:  3.8

In [12]:
idata_pymc3.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-01-26T18:38:57.552269
    inference_library:          pymc3
    inference_library_version:  3.8

In [13]:
idata_pymc3.prior

<xarray.Dataset>
Dimensions:  (chain: 1, draw: 150)
Coordinates:
  * chain    (chain) int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 142 143 144 145 146 147 148 149
Data variables:
    b0       (chain, draw) float64 -2.969 165.0 182.9 ... 170.4 173.4 177.4
    b1       (chain, draw) float64 -162.7 112.8 69.41 ... 175.2 321.5 75.85
    c0       (chain, draw) float64 0.3981 11.38 17.88 ... -6.508 -4.657 3.937
    c1       (chain, draw) float64 -12.84 32.65 1.721 ... 8.136 -13.4 -2.073
    b_sigma  (chain, draw) float64 216.8 98.68 421.0 76.42 ... 274.4 110.0 231.0
    c_sigma  (chain, draw) float64 6.678 3.207 2.96 2.228 ... 1.193 4.639 0.8392
Attributes:
    created_at:                 2020-01-26T18:38:57.547069
    inference_library:          pymc3
    inference_library_version:  3.8

In [14]:
idata_pymc3.predictions

<xarray.Dataset>
Dimensions:         (chain: 1, developer: 2, draw: 1600)
Coordinates:
  * chain           (chain) int64 0
  * draw            (draw) int64 0 1 2 3 4 5 6 ... 1594 1595 1596 1597 1598 1599
  * developer       (developer) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments  (chain, draw, developer) float64 8.781e+03 ... 8.201e+03
    github_commits  (chain, draw, developer) float64 27.64 39.84 ... 15.71 33.54
Attributes:
    created_at:                 2020-01-26T18:39:02.396739
    inference_library:          pymc3
    inference_library_version:  3.8

In [15]:
idata_pymc3.predictions_constant_data

<xarray.Dataset>
Dimensions:            (developer: 2)
Coordinates:
  * developer          (developer) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined  (developer) float64 3.6 5.1
Attributes:
    created_at:                 2020-01-26T18:39:02.397852
    inference_library:          pymc3
    inference_library_version:  3.8